In [13]:
import pandas as pd
import numpy as np

In [14]:
GDP = pd.read_csv('GDP.csv', encoding='cp1251')
population = pd.read_csv('Population.csv', encoding='cp1251')
GDP.head()

,CountryID,Country,GDP per capita (US dollars)
0,4,Afghanistan,611
1,8,Albania,"3,895"
2,12,Algeria,"4,163"
3,20,Andorra,"36,04"
4,24,Angola,"4,171"


In [15]:
population.head()

,CountryID,Year,Life expectancy (years)
0,4,2015,62.3
1,8,2015,77.7
2,12,2015,75.3
3,24,2015,60.2
4,28,2015,75.8


In [16]:
GDP_pop = pd.merge(GDP, population, on = 'CountryID', how = 'outer')
GDP_pop.head()

,CountryID,Country,GDP per capita (US dollars),Year,Life expectancy (years)
0,4,Afghanistan,611,2015.0,62.3
1,8,Albania,"3,895",2015.0,77.7
2,12,Algeria,"4,163",2015.0,75.3
3,20,Andorra,"36,04",NaN,NaN
4,24,Angola,"4,171",2015.0,60.2


подготовака окончена, приступим к выполнению